# Python Tutorial: Quickly Make Beautiful Interactive Maps
by Dr. Eyal Kazin, Senior Data Scientist at CA, London, UK


Nothing impresses a client or a colleague more than seeing their data on a meaningful plot. Even more so when it's on an intuitive map. Make it interactive, and they are likely to come back for more.    With the tools provided here you will be able to quickly create an interactive map that will leave an impression.  

All you need to do is install the latest version of [Folium](https://github.com/python-visualization/folium), a Python package that makes it easy to visualize data on an interactive map using the JavaScript [leaflet.js](http://leafletjs.com/) library.  

This is the first of a three part series:    
* **Part 1: The Basics**  
    * Creating a `Map` Object
    * Saving to HTML  
    * Location and Zoom
    * Map Tiles
    * Layer Control
    
    
* **Part 2: Zoom In**
    * Marker Groups
    * Marker Clusters  
    * Customising Markers
    * Circles
    
    
* **Part 3: Zoom Out**
    * Heatmaps: static or with temporal variation
    * Choropleths  
    * Customisation:  Beyond the Folium API
    * Summary: Putting it all together
    

**Disclaimer**  
We only discuss a handful of features which we use frequently. The reader is encouraged to use the module help pages to explore the full potential and updates in the API.  
(The Folium API used here assumes version `0.5.0`.)    

** Credits **  
Maps used are provided by © OpenStreetMap contributors.   
Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL  

This tutorial can be followed here or in [nbviewer](https://nbviewer.jupyter.org/github/SCLElections/folium_tutorial/blob/master/notebooks/beautiful_interactive_maps_part1.ipynb). All code is also avilable on [Github](https://github.com/SCLElections/folium_tutorial).  

In [1]:
import folium  # to install: pip install folium

folium.__version__

u'0.5.0'


## Testing the waters: Create a map with one line of code
A good starting point is to make sure that you can easily make a `Map` object and display it, either in your notebook or in HTML.  

In [2]:
map_ = folium.Map(control_scale=True)   

map_

Notes:
* The optional `control_scale=True` triggers the scale legend on the bottom left.  
* Use your mouse wheel or the buttons on the top left to zoom in and out.  

**Saving to HTML**

Alternatively you might want to save the map to an HTML file. Use the `save` module like this.

In [3]:
map_.save('my_map.html')

**Basic Trouble Shooting**

Note that if you get a blank image it might be due to at least one of the following:  
* In the case of blank image in a Jupyter notebook (but getting map in HTML):    
Verify that your Nbextensions ***Limit Output*** option is not limiting how much you can display.   
    E.g, if your settings is limited to 10000 characters you might get an error message simliar to: 
    ```
    limit_output extension: Maximum message size of 10000 exceeded with 20000 characters
    ```
* In the case of a blank image in both Jupyter and HTML, and/or missing layers:    
Some attributes passed might be corrupt (this is relevant for when passing text, e.g, when passing names to the objects)  

* In the case where the map is dispalying but a particular object/layer is not, verify that you passed the correct argument type.  
E.g, the Folium API can be inconsistent reguarding the type of geo points as tuples or lists.




Next we will use the API to determine the starting map extent.  

## Location, Location, Location (and Zoom)

The `Map` object takes keywords `location` (as in (latitude, longitude) tuple or list) and `zoom_start`.   
In the first map above we used the default highest zoom level (`zoom_start=1`), where in the following we use the at a fine granular level of `18`.  

In [4]:
center_latitude, center_longitude = 40.729183, -73.994263 # New York, NY, USA. Around Washington Square Park
location = (center_latitude, center_longitude)
zoom_start = 18

map_ = folium.Map(location=location, zoom_start=zoom_start, control_scale=True) 
map_

Note that if you want to limit the zoom range of the user, you can set these with `max_zoom` and `min_zoom`.  

## Map Tiles

Folium uses map tiles from [OpenStreetMap](https://www.openstreetmap.org) as its default, but you are free to choose others. 

Below we demonstrate using an alterative tiling called `positron` by [CartoDB](https://carto.com/), which is quite elegant  (they also have one called `dark_matter`!).    

We also demonstrate an interactive `LayerControl` feature, which allows users to select a tiling from a few options, as well as layers.  
The alternative tiles chosen here is call `Terrain` by [Stamen](http://maps.stamen.com/) and we add to the `Map` object via the `add_tile_layer` method.   

After creating the map toggle between the tiles using the radio buttons in the Layer Control.  


In [6]:
zoom_start = 14
map_ = folium.Map(location=location, zoom_start=zoom_start, tiles=None, control_scale=True)

# ===== Minimum required to use your own tile =====
tiles = 'CartoDB positron' # Notation: '[Source] [Tile]' (see table below for more examples)
tiles_name = 'CartoDB positron tiles'
map_.add_tile_layer(tiles=tiles, name=tiles_name)

# ===== Add this if you want more tile option for the user ======
more_tiles = 'Stamen Terrain'
more_tiles_name = 'Stamen Terrain tiles' # your choice of name for Layer Control (optional)
map_.add_tile_layer(tiles=more_tiles, name=more_tiles_name)

# ===== Layer Control ======
layer_control = folium.LayerControl(position='topleft', collapsed=False)
map_.add_child(layer_control) # adding Layer Control (optional). Alternatively: layer_control.add_to.(map_)

map_

Note: 
* The `LayerControl` feature appears on the top left. Try toggling between the tile options.   
* In theory, one could start up the folium map with a simple one-liner as below:  
```
map_ = folium.Map(location=location, zoom_start=zoom_start, tiles=tiles, control_scale=True)
```,    
however the API currently does not let you pass a `name` to the tile layer when using this approach.  

* `tiles` notation: '[Source] [Tiles]', e.g `'CartoDB positron'`. (See table below for more examples.)

**Free Tiles**

Some freely available tiles:  

| Source | Tiles |usage  `tiles=`|
|------|------|
| OpenStreetMap |   |`"OpenStreetMap"`| 
|Stamen|Terrain, Toner,  Watercolor|e.g, `"Stamen Terrain"`|
|CartoDB|positron, dark_matter|e.g, `"CartoDB dark_matter"`|


Alternatively you can pass a custom tileset. This requires a Leaflet-style URL to `tiles` that has the format `http://{s}.yourtiles.com/{z}/{x}/{y}.png`. 

** Tiles Terms of Use  **  
Distribution of maps created, of course, depends on your usage.     
This is our understanding:  

* **OpenStreetMap** - their [license](https://www.openstreetmap.org/copyright) indicates that you can freely use it for commercial purposes, as long as you credit it: © OpenStreetMap contributor  ([source: license page](https://opendatacommons.org/licenses/odbl/1.0/))  
* **Stamen Design** - their [main page](http://maps.stamen.com/#toner/12/37.7706/-122.3782) refers to [creativecommons.org license](https://creativecommons.org/licenses/by/3.0/) stating that one should cite:   
    * For Toner and Terrain: Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL  
    * For Watercolor: Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under CC BY SA    
* **CARTO**'s [End User License Agreement](https://drive.google.com/file/d/0B3OBExqwT6KJamRCcUhrWHJUQzQ/view) states:  
    *If you are on a Free Plan, you may make personal use of CARTO, but you are prohibited from using CARTO for
commercial purposes.*  


**Payed Tiles**    
Another source includes [Mapbox](https://www.mapbox.com/) which requires an API key for full access (keyword `API_key`). It has two freemiums with limited zooming: `Bright`, `Control Room` (no API key required).     
There is also mention of tiling by [Cloudmade](https://cloudmade.com/), which is [credited](https://wiki.openstreetmap.org/wiki/CloudMade) with the creation and first hosting of Leaflet (but has since moved to Github). It's not clear if this payed API is still functional.  


## Map Basics Summary

So far we have learned how to: 
* Initiate a `Map` object 
* Locate it 
* Choose its tiles  
* Add a Layer Control
* Display in a Jupyter notebook and save in HTML format.  

These could be summed up in three or four lines:    


In [7]:
map_ = folium.Map(location=(40.729183, -73.994263), zoom_start=14, tiles='Stamen Terrain', control_scale=True)
map_.add_tile_layer(tiles='CartoDB positron', name='CartoDB positron tiles') # optional
map_.add_child(folium.LayerControl(position='topleft', collapsed=True))  # adding Layer Control (optional).
# map_.save('my_map.html') # (optional) 
map_


In the subsequent parts you can learn how to add layers:  
* Part 2: Zoom In - Marker Groups and Icon Customisation.    
* Part 3: Zoom Out - Heatmaps, Choropleths and customising `folium.Element`s to add to a `Map` object.     